<a href="https://colab.research.google.com/github/gayathrimasil/gayathrimasil/blob/main/Frame_and_Bounding_Box_July23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Required Packages**

In [ ]:
!pip install torch torchvision
!pip install opencv-python-headless
!pip install pandas
!pip install tqdm

In [ ]:
!pip install --upgrade pillow


# **Clone Repository and Install Required Dependencies**

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5 # Navigate to Directory
!pip install -U -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 15821, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 15821 (delta 14), reused 26 (delta 10), pack-reused 15768
Receiving objects: 100% (15821/15821), 14.65 MiB | 14.33 MiB/s, done.
Resolving deltas: 100% (10826/10826), done.
[Errno 2] No such file or directory: 'yolov5 # Navigate to Directory'
/content/yolov5


# **Mount Google Drive**

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pillow


Found existing installation: Pillow 10.0.0
Uninstalling Pillow-10.0.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/PIL/*
    /usr/local/lib/python3.10/dist-packages/Pillow-10.0.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libXau-154567c4.so.6.0.0
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libbrotlicommon-702ab7f8.so.1
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libbrotlidec-7a796284.so.1.0.9
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libfreetype-8b16add1.so.6.20.0
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libharfbuzz-adaef2a2.so.0.60730.0
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libjpeg-dab02bf4.so.62.3.0
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/liblcms2-0821774a.so.2.0.15
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/liblzma-9dd5c569.so.5.4.3
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libopenjp2-20e347f0.so.2.5.0
    /usr/local/lib/python3.1

# **Install Ultralytics and Check Dependencies**

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.141 🚀 Python-3.10.6 torch-2.0.1+cu118 CPU (AMD EPYC 7B12)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 25.0/107.7 GB disk)


# **Import Required Libraries**

In [ ]:
import os
import cv2
import torch
import pandas as pd
from tqdm import tqdm
from pathlib import Path

# **Define paths**

In [7]:
wlasl_json = "/content/drive/MyDrive/SLR/new_json_csv.txt"
wlasl_videos = "/content/drive/MyDrive/SLR/sample Videos"
output_coordinates = "/content/drive/MyDrive/SLR/coordinates_with_details.csv"
wlasl_frames_folder = "/content/drive/MyDrive/SLR/frames_with_details"
yolov5_model_path = "/content/drive/MyDrive/SLR/yolov5/yolov5s.pt"

# **Load YoloV5 Model**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.hub.load("ultralytics/yolov5", "custom", path=yolov5_model_path).to(device)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

YOLOv5 🚀 2023-7-23 Python-3.10.6 torch-2.0.1+cu118 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


# **Load file to dataframe**

In [8]:
# Load CSV file into a DataFrame with 'videoID' column as text
df = pd.read_csv(wlasl_json, dtype={"videoID": str})


# **Frame Extraction, Object Detection, Bounding Box Coordinates**

In [9]:
# Helper function to perform object detection on a single frame
def detect_human_objects(image):
    results = model(image)
    detected_results = results.pandas().xyxy[0]
    return detected_results[detected_results["name"] == "person"]

# Process each line in the text file and update bounding box coordinates and extract frames
detected_boxes = []
for _, line in tqdm(df.iterrows(), total=len(df)):
    video_id = line['videoID']  # Video ID as string (keeping the leading zeros)
    video_filename = f"{video_id}.mp4"
    video_filepath = os.path.join(wlasl_videos, video_filename)

    if not os.path.exists(video_filepath):
        print(f"Warning: Video file '{video_filepath}' not found for video ID '{video_id}'.")
        continue

    # Create a folder for this video's frames
    video_folder = os.path.join(wlasl_frames_folder, video_id)
    os.makedirs(video_folder, exist_ok=True)

    # Open the video file
    cap = cv2.VideoCapture(video_filepath)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_start = line['frame_start']
    frame_end = line['frame_end']

    # Check if frame_start and frame_end are within the valid range of frame numbers
    if frame_start < 0:
        frame_start = 0
    if frame_end >= frame_count:
        frame_end = frame_count - 1

    for frame_num in range(frame_start, frame_end + 1):
        success, frame = cap.read()

        if not success:
            break

        # Convert BGR to RGB (YOLOv5 requires RGB images)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Save the frame as an image file in the video's folder
        frame_filename = f"frame{frame_num:04d}.jpg"
        frame_filepath = os.path.join(video_folder, frame_filename)
        cv2.imwrite(frame_filepath, frame_rgb)

        detected_results = detect_human_objects(frame_rgb)

        if len(detected_results) > 0:
            # Get the bounding box coordinates for the first detected person (you can modify this logic if needed)
            x_min, y_min, x_max, y_max = detected_results.iloc[0].values[:4]
            detected_boxes.append((video_id, line['signerID'], line['gloss'], line['fps'], frame_num, x_min, y_min, x_max, y_max, line['instance_id'], line['variation_id'], line['frame_end'], line['frame_start']))

    cap.release()

# Create DataFrame with detected bounding boxes and additional details
columns = ['videoID', 'signerID', 'gloss', 'fps', 'frame_num', 'x_min', 'y_min', 'x_max', 'y_max', 'instance_id', 'variation_id', 'frame_end', 'frame_start']
df_updated = pd.DataFrame(detected_boxes, columns=columns)

# Save updated DataFrame to a new CSV file
output_folder = os.path.dirname(output_coordinates)
os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist
df_updated.to_csv(output_coordinates, index=False)

print("Frame extraction, object detection, and CSV update completed!")


100%|██████████| 131/131 [01:30<00:00,  1.44it/s]

Frame extraction, object detection, and CSV update completed!
